# Getting data for Quito

For this case, data was collected manually

Source: http://www.quitoambiente.gob.ec/ambiente/index.php/descarga-datos-historicos

In [9]:
# libraries
import h3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [10]:
# read excel file with NO2 level
df=pd.read_excel('NO2.xlsx')
# read csv file with coordinates
dfCoordenates=pd.read_csv('coordenadas.csv',delimiter=';')

In [11]:
dfCoordenates['Station'] = dfCoordenates['Station'].str.upper()
dfCoordenates.set_index('Station',inplace=True)
# drop some stations without data
dfCoordenates.drop(['TUMBACO', 'JIPIJAPA'],inplace=True)
dfCoordenates.index.name = None
# get h3id for each station
dfCoordenates['h3id'] = dfCoordenates.apply(lambda row: h3.geo_to_h3(row.Latitude, row.Longitude, 9), axis=1)

h3dict = dfCoordenates.h3id.to_dict()

In [12]:
# remove first row
df.drop([0],inplace=True)

# Rename column unnamed
df.rename(columns={df.columns[0] :'Date'}, inplace=True )

# convert the 'Date' column to datetime format
df['Date']= pd.to_datetime(df['Date'])

# convert other columns to numeric
df[df.columns[1:]] = df[df.columns[1:]].apply(pd.to_numeric, errors='coerce', axis=1)

### For all cases analyzed data is for April 2019

In [13]:
# April 2019
i = pd.date_range('2019-04-01 00:00:00', periods=31, freq='24H')
dfApril2019=df.loc[(df.Date >= i[0]) & (df.Date <= i[-1])]
# Drop last row (1/5/2019)
dfApril2019.drop([134377],inplace=True)
# Drop TUMBACO, CONDADO, TURUBAMBA E CHILLOGALLO (empty ones)
dfApril2019.drop(columns=['TUMBACO', 'CONDADO', 'TURUBAMBA' ,'CHILLOGALLO'],inplace=True)

In [14]:
df = dfApril2019.melt(id_vars='Date')
df['h3id'] = df.variable.apply(lambda s: h3dict[s])
# convert ug/m3 to ppb
# The conversion assumes an ambient pressure of 1 atmosphere and a temperature of 25 degrees Celsius.
# https://www2.dmu.dk/atmosphericenvironment/expost/database/docs/ppm_conversion.pdf
df['NO2']=df['value']/1.88
df['time'] = pd.to_datetime(df.Date)
# save csv with data
df[['h3id', 'time', 'NO2']].to_csv('y_data.csv', index=False)